In [ ]:
from arg_utils import is_notebook, get_cfg
cfg = get_cfg()
# choices
classical_models = ["linear", "xgb", "rf"]
nn_models = ["mlp", "rnn", "transformer", "clip", "gpt"]
# override variables to experiment in notebook
if is_notebook():    
    cfg["target_name"] = "ICP_Vital"   # ICP_Vital" , long_icp_hypertension_2
    cfg["db_name"] = "UKE"  # "UKE", "MIMIC"
    cfg["minutes"] = 60
    cfg["model_type"] = "gpt"
    
    # do experiments on:fill_type, target_nan_quantile, train_noise_std, 
    #  min_len(increase from 20 to higher), grad_clip_val (at 1 so far), weight_decay (at 0.2 so far)
    
    cfg["fill_type"] = "median" # "pat_mean", "median", "pat_ema" "pat_ema_mask"
    cfg["norm_method"] = None # z, or none

    
    cfg["bs"] = 32 # 8 best for rnn, 32 for GPT
    cfg["max_len"] = 128
    cfg["min_len"] = 128
    cfg["target_nan_quantile"] = 0.9999
    cfg["block_size"] = 0

    # classical model args
    cfg["flat_block_size"] = 8
    # general args
    cfg["max_epochs"] = 20
    cfg["use_nan_embed"] = False
    cfg["weight_decay"] = 0.2
    
    
    # rnn params
    cfg["hidden_size"] = 2048
    cfg["rnn_type"] = "gru"
    
    # transformer params
    cfg["mode"] = "train_mlp_norm"  # "adapters", "train_mlp_norm",  "train_norm", "freeze" (does not train)
    
    cfg["gpu"] = 1

    cfg["tune_hebo"] = False
    cfg["opt_steps"] = 5
    
    
# overrides and calculated default vals
if cfg["lr"] is None:
    model_type = cfg["model_type"]
    if model_type == "clip":
        cfg["lr"] = 0.001
    elif model_type == "gpt":
        # bs 8 and gpt2 take 9.8GB with max seq len of 512
        # bs 16 with max seq len of 256
        # bs 32 with max seq len 128 only 7.4GB, good performance and fast - 6.9 if mlp_norm
        # bs 64 with len 128 and mlp_norm = 10.9GB. 9.4GB for freeze
        cfg["lr"] = 0.00005
    else:
        cfg["lr"] = 0.0001  # 0.01 works kind of for nan_embed
        
#cfg["val_check_interval"] = int(cfg["val_check_interval"] * (32 / cfg["batch_size"]))
    

import os
os.environ["CUDA_VISIBLE_DEVICES"] = str(cfg["gpu"])
# disable wandb printing
os.environ['WANDB_SILENT'] = "true"

import torch
torch.multiprocessing.set_sharing_strategy('file_system')
locals().update(cfg)

In [ ]:
import pandas as pd


import logging
import pytorch_lightning
logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)
pytorch_lightning.utilities.distributed.log.setLevel(logging.ERROR)

In [ ]:
# load df
path = f"data/DB_{cfg['db_name']}_{cfg['minutes']}_final_df.pkl"
df = pd.read_pickle(path)

In [ ]:
from tune_hebo import get_hebo_space, tune_hebo

# run hebo tuning if wanted
if cfg["tune_hebo"]:
    space = get_hebo_space(cfg["model_type"])
    tune_hebo(space, cfg)


In [ ]:
# OPTUNA
import optuna
import pandas as pd
from tune_utils import objective_optuna


study = optuna.create_study()  # Create a new study.
study.optimize(lambda study: objective_optuna(study, df, cfg), 
               n_trials=cfg["opt_steps"])  # Invoke optimization of the objective_optuna function.

In [ ]:
from tune_utils import make_optuna_foldername, save_study_results

# create a folder to save the results
folder_name = make_optuna_foldername(cfg)
save_study_results(study, folder_name, cfg["model_type"])


In [ ]:
from tune_utils import retrain_best_trial

val_scores, test_scores, weights = retrain_best_trial(study, df, cfg)

In [ ]:
from soup_utils import create_and_eval_soup
from tune_utils import merge_params_in_cfg, setup_dm

soup_cfg = merge_params_in_cfg(cfg, study.best_params)
soup_dm = setup_dm(df, soup_cfg)

# make soup of different seeds of the best hyperparam set
soup_val_score, soup_test_score = create_and_eval_soup(soup_dm, soup_cfg, weights)
# save scores of soup model
soup_scores_df = pd.DataFrame({"val_score": [soup_val_score], "test_score": [soup_test_score]})
soup_scores_df.to_csv(f"{folder_name}/best_param_seed_soup_scores.csv")

In [ ]:
# get params of some of the best optuna trials to create a soup from
from tune_utils import get_best_params, train_multiple

top_params, top_vals = get_best_params(study, num_trials=5)
top_param_val_scores, top_param_test_scores, top_param_weights = train_multiple(top_params, df, cfg)

In [ ]:
# soup of some of the best hyperparameter sets
soup_val_score, soup_test_score = create_and_eval_soup(soup_dm, soup_cfg, top_param_weights[:5])
# save scores of soup model
soup_scores_df = pd.DataFrame({"val_score": [soup_val_score], "test_score": [soup_test_score]})
soup_scores_df.to_csv(f"{folder_name}/top_param_soup_scores.csv")